In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("").setMaster("spark://workspace:7077")
sparkContext = SparkContext(conf=conf)
spark = SparkSession(sparkContext=sparkContext)

23/12/30 17:09:55 WARN Utils: Your hostname, workspace resolves to a loopback address: 127.0.0.1; using 220.118.158.128 instead (on interface eno1)
23/12/30 17:09:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 17:09:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import expr, explode, col

In [3]:
json_dir = "file:///home/hooniegit/git/Spotify-DemoProject/fastapi/data/artists/2023-12-29"

In [4]:
df = spark.read.option("multiline", "true").json(json_dir) \
    .select(explode('artists').alias('artists')) \
    .withColumn("image", explode(expr("artists.images"))) \
    .filter(col("image.height") == 640) \
    .withColumn("artist_id", expr("artists.id")) \
    .withColumn("artist_name", expr("artists.name")) \
    .withColumn("genre", explode(expr("artists.genres"))) \
    .withColumn("followers", expr("artists.followers.total")) \
    .withColumn("artist_image_url", expr("image.url")) \
    .select("artist_id", "artist_name", "genre", "artist_image_url", "followers")
df.show()

+--------------------+-------------+--------------------+--------------------+---------+
|           artist_id|  artist_name|               genre|    artist_image_url|followers|
+--------------------+-------------+--------------------+--------------------+---------+
|3qsKSpcV3ncke3hw5...|   Young Miko|         trap latino|https://i.scdn.co...|  1946884|
|3qsKSpcV3ncke3hw5...|   Young Miko|       urbano latino|https://i.scdn.co...|  1946884|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|          indie rock|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|            la indie|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|modern alternativ...|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|         modern rock|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|          pov: indie|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|    stomp and holler|https://i.scdn.co...|  1156962|
|4B8u7XuCSCPhqrVoz...

In [5]:
spark.stop()